In [1]:
# This notebook contains the code to (re)-generate all our standardized temperature tables
# from the raw data we got.
#
# Raw data come from two sources:
# 1) H/He tables come from Allona Vazan.
# 2) Fe/SiO2/H2O tables come from ____. 



# WARNING: THIS NOTEBOOK IS ONLY MEANT TO BE RUN IF THE LIBRARY IS DOWNLOADED LOCALLY!


In [2]:
import csv
import math
import numpy                  as np

from lib.temperature import TemperatureTable

In [3]:
TEMP_RANGE = np.arange(1.06, 7.06, 0.08)
PRESSURE_RANGE = np.arange(-6,19,0.2)
DENSITY_RANGE = np.arange(-14,3.2,0.05)


# Density of two mixtures at same temp/pressure. 
def get_rho(rho1, rho2, mix1, mix2):
    return 1.0/(mix1/rho1 + mix2/rho2)

In [4]:
# Code to load/parse/combine Allona's files.
           
def load_temp_pressure_alona_file(filename):    
    fp = open(filename)
    data = fp.readlines()
    retval = []
    row_idx = 0
    
    while row_idx < len(data):
        # The header has the log temperature and the number of lines.
        temp, n_rows = [x for x in data[row_idx].split()]
        temp = float(temp)
        n_rows=int(n_rows)
        for i in range(n_rows):
            row = [x for x in data[row_idx+1+i].split()]
            pressure = float(row[0])
            rho = float(row[3])
            retval.append((temp, pressure, rho))
        row_idx += n_rows + 1
    
    return dict([((float(x[0]), float(x[1])), float(x[2])) for x in retval])
            
def combine_temp_pressure_alona_files(file1, file2, pct1, pct2, outname, composition=0):
    comp_1 = load_temp_pressure_alona_file(file1)
    comp_2 = load_temp_pressure_alona_file(file2)
    data = []
    for temp in TEMP_RANGE:
        for pressure in PRESSURE_RANGE:
            temp = round(temp, 2)
            pressure = round(pressure, 1)
            if (temp, pressure) in comp_1 and (temp, pressure) in comp_2:
                # need to compute new rho.
                rho_1 = 10**comp_1[(temp, pressure)]
                rho_2 = 10**comp_2[(temp, pressure)]
                
                rho = get_rho(rho_1, rho_2, mix1=pct1, mix2=pct2)
                data.append((temp, pressure, np.log10(rho)))
                
    with open(outname,"w") as fp:
        # Write header, just for clarification
        fp.write("Composition = %s\n"%composition)
        fp.write("log temp (K), log pressure (dyne/cm^2), logdensity (g/cc)\n")
        for d in data:
            fp.write(",".join(map(lambda x : str(x),d))+"\n")

In [5]:
# Code to read "newtab_AV" files.

def get_AV_table(filename, include_entropy, start_row=7):
    data = [i.strip('\n').split() for i in open(filename)][start_row:]
    temp = [float(x[0]) for x in data]
    density = [float(x[1]) for x in data]
    pressure = [float(x[2]) for x in data]

    # Convert units

    # Temperature is in Kev, so mutiple by 1.16e7
    temp = 10**np.array(temp)*1.1605e7

    # Density is in gr/cc, so no conversion
    density = 10**np.array(density)

    # Pressure is in krk/cc, so mupltiply by 1e16 (I think?)
    pressure = 10**np.array(pressure)*1e16

    t_table = TemperatureTable(0, filename, temp, pressure, density)
    
    if include_entropy:
        # We're going to fake things just create a t-table where "density" is the entropy column (I believe the last column)
        entropy =  [float(x[4]) for x in data]
        entropy_table = TemperatureTable(0, filename, temp, pressure, entropy)
        return t_table, entropy_table
    
    return t_table
    
def create_temp_pressure_from_AV_table(input_name, output_name, composition = 0, add_entropy = False, start_row=7):
    if add_entropy:
        av_table, ev_table = get_AV_table(input_name, True, start_row)
    else:
        av_table = get_AV_table(input_name, False)

    data = []
    for temp in TEMP_RANGE:
        for pressure in PRESSURE_RANGE:
            temp = round(temp, 2)
            pressure = round(pressure, 1)
            density = av_table.get_density(10**temp, 10**pressure)
            if density is not None:
                if add_entropy:
                    entropy = ev_table.get_density(10**temp, 10**pressure)
                    data.append((temp, pressure, np.log10(density), np.log10(entropy)))
                else:
                    data.append((temp, pressure, np.log10(density)))
    
    with open(output_name,"w") as fp:
        # Write header, just for clarification
        fp.write("Composition = %s\n"%composition)
        if add_entropy:
            fp.write("log(temp), log pressure (dyne/cm^2), log density (g/cc), log entropy (log os jrk/g*kev)\n")
        else:
            fp.write("log(temp), log pressure (dyne/cm^2), log density (g/cc)\n")
        for d in data:
            fp.write(",".join(map(lambda x : str(x),d))+"\n")


In [6]:
# This section is for combining files in our own format to create new compositions.

# All the tables we create have the same ranges, so rather than 
# go through the TemperatureTable again, just go through the range
# and create composite densities if needed.

def load_temp_pressure_file(filename):
    data = [i.strip('\n').split(',') for i in open(filename)][2:]
    return dict([((float(x[0]), float(x[1])), float(x[2])) for x in data])

def combine_temp_pressure_files(file1, file2, pct1, pct2, outname, composition=0):
    comp_1 = load_temp_pressure_file(file1)
    comp_2 = load_temp_pressure_file(file2)
    data = []
    for temp in TEMP_RANGE:
        for pressure in PRESSURE_RANGE:
            temp = round(temp, 2)
            pressure = round(pressure, 1)
            if (temp, pressure) in comp_1 and (temp, pressure) in comp_2:
                # need to compute new rho.
                rho_1 = 10**comp_1[(temp, pressure)]
                rho_2 = 10**comp_2[(temp, pressure)]
                
                rho = get_rho(rho_1, rho_2, mix1=pct1, mix2=pct2)
                data.append((temp, pressure, np.log10(rho)))
                
    with open(outname,"w") as fp:
        # Write header, just for clarification
        fp.write("Composition = %s\n"%composition)
        fp.write("log temp, log pressure (dyne/cm^2), logdensity (g/cc)\n")
        for d in data:
            fp.write(",".join(map(lambda x : str(x),d))+"\n")


In [7]:
# These actually create the various files. 
# Uncomment as needed.

create_temp_pressure_from_AV_table("data/raw_files/COtable.txt", "data/CO_temp_pressure_to_density.txt", start_row=0)
create_temp_pressure_from_AV_table("data/raw_files/H2O_newtab_AV.dat", "data/water_temp_pressure_to_density.txt")
create_temp_pressure_from_AV_table("data/raw_files/H2O_newtab_AV.dat", "data/water_temp_pressure_to_density_with_entropy.txt", add_entropy=True)
create_temp_pressure_from_AV_table("data/raw_files/Fe_newtab_AV.dat", "data/Fe_temp_pressure_to_density.txt")
create_temp_pressure_from_AV_table("data/raw_files/SiO2_newtab_AV.dat", "data/SiO2_temp_pressure_to_density.txt")
combine_temp_pressure_alona_files("data/raw_files/Htable.txt", "data/raw_files/Hetable.txt", 0.725, 0.275, "data/env_temp_pressure_to_density.txt")
combine_temp_pressure_alona_files("data/raw_files/Htable.txt", "data/raw_files/Hetable.txt", 1.0, 0.0, "data/H_temp_pressure_to_density.txt")
combine_temp_pressure_alona_files("data/raw_files/Htable.txt", "data/raw_files/Hetable.txt", 0.0, 1.0, "data/He_temp_pressure_to_density.txt")
combine_temp_pressure_alona_files("data/raw_files/Htable.txt", "data/raw_files/Hetable.txt", 0.72, 0.28, "data/allona_env_temp_pressure_to_density.txt")
combine_temp_pressure_files("data/water_temp_pressure_to_density.txt", "data/env_temp_pressure_to_density.txt", 0.9, 0.1, "data/Z90_temp_pressure_to_density.txt", "90% Water, 10% env")
combine_temp_pressure_files("data/water_temp_pressure_to_density.txt", "data/env_temp_pressure_to_density.txt", 0.95, 0.05, "data/Z95_temp_pressure_to_density.txt", "95% Water, 5% env")
combine_temp_pressure_files("data/Fe_temp_pressure_to_density.txt", "data/SiO2_temp_pressure_to_density.txt", 0.24, 0.76, "data/dunite_temp_pressure_to_density.txt", "24% Fe, 76% SiO2")
combine_temp_pressure_files("data/dunite_temp_pressure_to_density.txt", "data/water_temp_pressure_to_density.txt", 0.65, 0.35, "data/Z_temp_pressure_to_density.txt", "65% Dunite, 35% Water")
combine_temp_pressure_files("data/SiO2_temp_pressure_to_density.txt", "data/water_temp_pressure_to_density.txt", 0.667, 0.333, "data/sio2_water_2_1_temp_pressure_to_density.txt", "66% Sio2, 33% Water")
combine_temp_pressure_files("data/SiO2_temp_pressure_to_density.txt", "data/water_temp_pressure_to_density.txt", 0.725, 0.275, "data/sio2_water_3_1_temp_pressure_to_density.txt", "75% Sio2, 25% Water")
combine_temp_pressure_files("data/SiO2_temp_pressure_to_density.txt", "data/water_temp_pressure_to_density.txt", 0.8, 0.2, "data/sio2_water_4_1_temp_pressure_to_density.txt", "80% Sio2, 20% Water")